In [37]:
import numpy as np
from typing import List, Tuple
from collections import deque

In [5]:
def parse_input(input_file: str) -> np.array:
    with open(input_file, "r") as file:
        char_array = None

        for line in file:
            line_chars = list(line.strip()) 
            line_array = np.array(line_chars)

            # first line: assignment
            if char_array is None:
                char_array = line_array
            # 2nd to nth line: append to existing 2d-array
            else:
                char_array = np.vstack((char_array, line_array))

    #print(char_array)
    return char_array

In [6]:
parse_input('example.txt')

array([['7', '-', 'F', '7', '-'],
       ['.', 'F', 'J', '|', '7'],
       ['S', 'J', 'L', 'L', '7'],
       ['|', 'F', '-', '-', 'J'],
       ['L', 'J', '.', 'L', 'J']], dtype='<U1')

In [7]:
# | is a vertical pipe connecting north and south.
# - is a horizontal pipe connecting east and west.
# L is a 90-degree bend connecting north and east.
# J is a 90-degree bend connecting north and west.
# 7 is a 90-degree bend connecting south and west.
# F is a 90-degree bend connecting south and east.

In [8]:
def get_next_direction(pipe_array: np.array, entry_direction: str, row_index: int, col_index: int) -> Tuple[int, int]:
    current_pipe = pipe_array[row_index, col_index]
    #print(f"Action, current_pipe: {current_pipe}, entry_direction: {entry_direction}, current_row_index: {row_index}, current_col_index: {col_index}")
    if current_pipe == "|" and entry_direction == "up":
        return "up"
    elif current_pipe == "|" and entry_direction == "down":
        return "down"
    elif current_pipe == "-" and entry_direction == "right":
        return "right"
    elif current_pipe == "-" and entry_direction == "left":
        return "left"
    elif current_pipe == "L" and entry_direction == "down":
        return "right"
    elif current_pipe == "L" and entry_direction == "left":
        return "up"
    elif current_pipe == "J" and entry_direction == "right":
        return "up"
    elif current_pipe == "J" and entry_direction == "down":
        return "left"
    elif current_pipe == "7" and entry_direction == "up":
        return "left"
    elif current_pipe == "7" and entry_direction == "right":
        return "down"
    elif current_pipe == "F" and entry_direction == "up":
        return "right"
    elif current_pipe == "F" and entry_direction == "left":
        return "down"
    else:
        print(f"Error, current_pipe: {current_pipe}, entry_direction: {entry_direction}, current_row_index: {row_index}, current_col_index: {col_index}")

In [9]:
def solve_part1(input_file: str) -> int:
    pipe_array = parse_input(input_file)
    pipe_array = np.pad(pipe_array, pad_width=1, constant_values='.')
    #print(pipe_array)
    count_array = np.zeros(pipe_array.shape, dtype=int) - 1
    
    row_indices, col_indices = np.where(pipe_array == 'S')
    start_location = np.array([row_indices[0], col_indices[0]])

    count_array[start_location[0], start_location[1]] = 0

    move_difference = {
        "up": (-1, 0),
        "down": (1, 0),
        "left": (0, -1),
        "right": (0, 1)
    }

    # get neighbor field of S (look at this to find out if your S is also a "F") adjust first direction to your input
    # start_field = pipe_array[start_location[0]-1:start_location[0]+2, start_location[1]-1:start_location[1]+2]
    # print(start_field)
    # start field in my example is "F", so I start by walking "right". 
    # You could also walk down I suppose and walk the circle in the other direction
    current_location = start_location + move_difference["right"]
    current_direction = "right"
    current_count = 1
    count_array[current_location[0], current_location[1]] = current_count
    # traverse loop
    while pipe_array[current_location[0], current_location[1]] != "S":
        current_direction = get_next_direction(pipe_array, current_direction, current_location[0], current_location[1])
        current_location = current_location + move_difference[current_direction]
        current_count += 1
        count_array[current_location[0], current_location[1]] = current_count

    print("Loop length", current_count)
    return current_count//2

In [10]:
solve_part1('input.txt')

Loop length 14194


7097

In [34]:
# Define the patterns for each character
patterns = {
    'J': np.array([['.', 'O', '.'], 
                   ['O', 'O', '.'], 
                   ['.', '.', '.']]),
    'L': np.array([['.', 'O', '.'], 
                   ['.', 'O', 'O'], 
                   ['.', '.', '.']]),
    '7': np.array([['.', '.', '.'], 
                   ['O', 'O', '.'], 
                   ['.', 'O', '.']]),
    'F': np.array([['.', '.', '.'], 
                   ['.', 'O', 'O'], 
                   ['.', 'O', '.']]),
    '|': np.array([['.', 'O', '.'], 
                   ['.', 'O', '.'], 
                   ['.', 'O', '.']]),
    '-': np.array([['.', '.', '.'], 
                   ['O', 'O', 'O'], 
                   ['.', '.', '.']]),
    '.': np.array([['.', '.', '.'], 
                   ['.', '#', '.'], 
                   ['.', '.', '.']])
}

# Create a function to expand the array
def expand_main_pipes(pipe_array: np.array, patterns) -> np.array:
    # Initialize the expanded array with the correct size
    expanded_array = np.empty((3 * pipe_array.shape[0], 3 * pipe_array.shape[1]), dtype=str)
    
    # Iterate through each entry in the original array
    for i in range(pipe_array.shape[0]):
        for j in range(pipe_array.shape[1]):
            # Get the 3x3 pattern for the current entry
            pattern = patterns.get(pipe_array[i, j], patterns['.'])
            # Place the pattern in the expanded array
            expanded_array[3*i:3*i+3, 3*j:3*j+3] = pattern
            
    return expanded_array


In [38]:
def discard_outside_values(ext_pipe_array: np.array) -> np.array:
        # Initialize a visited array of the same shape
    visited = np.zeros_like(ext_pipe_array, dtype=bool)

    # Initialize the queue with all edge coordinates
    queue = deque()
    for i in range(ext_pipe_array.shape[0]):
        queue.append((i, 0))
        queue.append((i, ext_pipe_array.shape[1] - 1))
    for j in range(1, ext_pipe_array.shape[1] - 1):
        queue.append((0, j))
        queue.append((ext_pipe_array.shape[0] - 1, j))

    # Perform depth-first search
    while queue:
        x, y = queue.popleft()

        # Check if current position is within bounds and not visited
        if 0 <= x < ext_pipe_array.shape[0] and 0 <= y < ext_pipe_array.shape[1] and not visited[x, y]:
            visited[x, y] = True

            # Add neighbors to the queue if they are not 'O'
            if ext_pipe_array[x, y] != 'O':
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < ext_pipe_array.shape[0] and 0 <= ny < ext_pipe_array.shape[1]:
                        queue.append((nx, ny))

    # Apply the visited mask to the original array (remove all # values not inside the loop)
    ext_pipe_array[visited] = '.'

    return ext_pipe_array

In [53]:
def solve_part2(input_file: str) -> int:
    pipe_array = parse_input(input_file)
    pipe_array = np.pad(pipe_array, pad_width=1, constant_values='.')
    #print(pipe_array)
    mark_array = np.full(pipe_array.shape, ".", dtype=str)
    
    row_indices, col_indices = np.where(pipe_array == 'S')
    start_location = np.array([row_indices[0], col_indices[0]])

    # note down all fields of the main loop (to discard all pipes not in the main loop)
    # replace value here with what your "S" represents (see below code to check your start_field visually)
    mark_array[start_location[0], start_location[1]] = "F"
    

    move_difference = {
        "up": (-1, 0),
        "down": (1, 0),
        "left": (0, -1),
        "right": (0, 1)
    }

    # get neighbor field of S (look at this to find out if your S is also a "F") adjust first direction to your input
    start_field = pipe_array[start_location[0]-1:start_location[0]+2, start_location[1]-1:start_location[1]+2]
    print(start_field)
    # start field in my example is "F", so I start by walking "right". 
    # You could also walk down I suppose and walk the circle in the other direction
    current_location = start_location + move_difference["right"]
    current_direction = "right"
    # traverse loop
    while pipe_array[current_location[0], current_location[1]] != "S":
        mark_array[current_location[0], current_location[1]] = pipe_array[current_location[0], current_location[1]]
        current_direction = get_next_direction(pipe_array, current_direction, current_location[0], current_location[1])
        current_location = current_location + move_difference[current_direction]

    #print(mark_array)
    #print(mark_array[1:-1, 1:-1])
    np.savetxt("mark_array.txt", mark_array[1:-1, 1:-1], fmt="%s")

    expanded_array = expand_main_pipes(mark_array[1:-1, 1:-1], patterns)
    #print(expanded_array)
    np.savetxt("expanded_array.txt", expanded_array, fmt="%s")

    cleaned_expanded_array = discard_outside_values(expanded_array)
    #print(cleaned_expanded_array)
    np.savetxt("cleaned_expanded_array.txt", cleaned_expanded_array, fmt="%s")
    
    # Count the occurrences of "#"
    count_of_tiles = np.count_nonzero(cleaned_expanded_array == "#")

    return count_of_tiles

In [56]:
solve_part2('example.txt')

[['.' '.' 'F']
 ['.' 'S' 'J']
 ['.' '|' 'F']]


1